In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import zipfile
import requests
from io import BytesIO
import seaborn as sns
from matplotlib.gridspec import GridSpec
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
accuracy_score,
precision_score,
recall_score,
f1_score,
fbeta_score,
confusion_matrix,
)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
import random
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
import itertools
import torch.nn.init as init
from sklearn.model_selection import cross_val_score, cross_validate, KFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import time
from torchvision.models import resnet50

import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier

In [31]:
df = pd.read_csv('sample_test_dataset.csv')

# Split the DataFrame into X_test and y_test
X_test = df.iloc[:, :-1]  # Select all columns except the last one
y_test = df.iloc[:, -1]   # Select the last column

In [34]:
import torch
import torch.nn as nn
import pandas as pd
from torch.nn import init

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_hidden_layers, dropout_rate, activation, batch_norm):
        super(NeuralNetwork, self).__init__()
        self.fc_input = nn.Linear(input_size, hidden_size)
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_size, hidden_size) for _ in range(num_hidden_layers)])
        self.fc_output = nn.Linear(hidden_size, num_classes)

        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()

        self.dropout = nn.Dropout(dropout_rate)
        if batch_norm:
            self.batchnorms = nn.ModuleList([nn.BatchNorm1d(hidden_size) for _ in range(num_hidden_layers)])
        self.batch_norm = batch_norm

    def forward(self, x):
        out = self.fc_input(x)
        out = self.activation(out)
        out = self.dropout(out)

        for i, layer in enumerate(self.hidden_layers):
            out = layer(out)
            out = self.batchnorms[i](out) if self.batch_norm else out
            out = self.activation(out)
            out = self.dropout(out)

        out = self.fc_output(out)
        return out


def init_weights(m):
    if isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0.0)

df = pd.read_csv('sample_test_dataset.csv')

# Split the DataFrame into X_test and y_test
X_test = df.iloc[:, :-1]  # Select all columns except the last one
y_test = df.iloc[:, -1]   # Select the last column
# Load the pretrained model
hidden_size = 64
num_hidden_layers = 2
batch_norm = False
activation = 'relu'
dropout_rate = 0.1
input_size = X_test.shape[1]
num_classes = 2
pretrained_model = NeuralNetwork(input_size, hidden_size, num_classes, num_hidden_layers, dropout_rate, activation, batch_norm)
pretrained_model.load_state_dict(torch.load("DNN_model.pt"),strict=True)


<All keys matched successfully>

In [ ]:
# Assuming you have already loaded the pretrained model as "pretrained_model"

pretrained_model.eval()  # Set the model in evaluation mode

test_losses = []
test_accuracies = []

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_inputs = test_inputs.to(device)
        test_labels = test_labels.to(device)
        test_outputs = pretrained_model(test_inputs)
        test_loss = criterion(test_outputs, test_labels).item()
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_accuracy = torch.sum(test_predicted == test_labels).item() / len(test_labels)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

average_test_loss = sum(test_losses) / len(test_losses)
average_test_accuracy = sum(test_accuracies) / len(test_accuracies)

print(f"Average Test Loss: {average_test_loss:.4f}")
print(f"Average Test Accuracy: {average_test_accuracy * 100:.2f}%")


In [ ]:

# Set the model in evaluation mode
pretrained_model.eval()

# Use the pretrained model for predictions or further training

In [40]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [44]:
test_loader = DataLoader(test_dataset, batch_size=batch_size)
device = 'cpu'

In [48]:
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)
y_test_encoded = label_encoder.transform(y_test)

y_test_encoded = label_encoder.transform(y_test)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)
criterion = nn.CrossEntropyLoss()
model = pretrained_model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    test_correct = 0
    test_samples = 0
    true_labels = []
    predicted_labels = []

    for test_inputs, test_labels in test_loader:
        test_inputs = test_inputs.to(device)
        test_labels = test_labels.to(device)
        test_outputs = model(test_inputs)
        test_loss += criterion(test_outputs, test_labels).item()
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_correct += torch.sum(test_predicted == test_labels).item()
        test_samples += test_labels.size(0)
        true_labels.extend(test_labels.tolist())
        predicted_labels.extend(test_predicted.tolist())

    test_loss /= test_samples
    test_accuracy = test_correct / test_samples
    cm = confusion_matrix(true_labels, predicted_labels)
    tn, fp, fn, tp = cm.ravel()
    if (tp + fp) != 0:
        precision = tp / (tp + fp)
    else:
        precision = 0.0
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    accuracy = accuracy_score(true_labels, predicted_labels)
input_size = (X_test.shape[1],)



print(f'Test Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1 Score: {f1 * 100:.2f}%')


Test Accuracy: 84.16%
Precision: 82.74%
Recall: 86.36%
F1 Score: 84.51%
